<a href="https://colab.research.google.com/github/girish445ai/Recurrent_Neural_networks/blob/main/SeqtoSeq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
import matplotlib.pyplot as plt
from random import shuffle
import pandas as pd
import keras
from keras.layers import Input, LSTM, Dense, Embedding, GRU, SimpleRNN, Dropout, Activation, dot, concatenate, TimeDistributed
from keras.models import Model

!pip3 install tensorflow -qqq
!pip3 install wandb -qqq
import wandb
!wandb login
from wandb.keras import WandbCallback

     |████████████████████████████████| 462 kB 7.0 MB/s 
     |████████████████████████████████| 1.8 MB 7.8 MB/s 
     |████████████████████████████████| 181 kB 48.5 MB/s 
     |████████████████████████████████| 144 kB 53.3 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [2]:
!wget -nc https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar

--2022-04-15 14:50:27--  https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.141.128, 2607:f8b0:4023:c0b::80
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2008340480 (1.9G) [application/x-tar]
Saving to: ‘dakshina_dataset_v1.0.tar’

dakshina_dataset_v1 100%[===================>]   1.87G   123MB/s    in 13s     

2022-04-15 14:50:39 (153 MB/s) - ‘dakshina_dataset_v1.0.tar’ saved [2008340480/2008340480]



In [4]:
test_path = 'te.translit.sampled.test.tsv'
val_path = 'te.translit.sampled.dev.tsv'
train_path = 'te.translit.sampled.train.tsv'